In [2]:
from qutip import sigmax, sigmay, sigmaz, identity, qeye, tensor, \
                  Qobj, simdiag, basis, commutator, rand_ket, expect
from functools import reduce
import numpy as np
import importlib
import os
import time 

# import custom module
import spin_chain
importlib.reload(spin_chain); 

ModuleNotFoundError: No module named 'spin_chain'

In [ ]:
# define Pauli matrices 
σ_x = sigmax()
σ_y = sigmay()
σ_z = sigmaz()
I2 = qeye(2)
I16 = qeye([2]*4)

def quantities(N, Jx, Jy, Jz, tolerance):

    
    if N % 2 != 0: 
        raise ValueError("Please enter an even number of sites")
    
    # define Pauli matrices and constants
    σ_x = sigmax()
    σ_y = sigmay()
    σ_z = sigmaz()
    π = np.pi

    # Interaction coefficients, which we assume are uniform throughout the lattice
    Jx_list = Jx*np.ones(N)
    Jy_list = Jy*np.ones(N)
    Jz_list = Jz*np.ones(N)

    # Setup operators for individual qubits; 
    # here sx_list[j] = X_j, sy_list[j] = Y_j, and sz_list[j] = Z_j
    # since the Pauli matrix occupies the jth location in the tensor product of N terms
    # of which N-1 terms are the identity
    sx_list, sy_list, sz_list = [], [], []

    for i in range(N):
        op_list = [qeye(2)]*N
        op_list[i] = σ_x
        sx_list.append(tensor(op_list))
        op_list[i] = σ_y
        sy_list.append(tensor(op_list))
        op_list[i] = σ_z
        sz_list.append(tensor(op_list))

    # define variable for total Hamiltonian H_N and the list of all local 
    # Hamiltonians H_list
    H_N = 0 
    H_list = []
    
    # collect 
    for j in range(N - 1):

        # find H_ij, the Hamiltonian between the ith and jth sites 
        H_ij = Jx_list[j] * sx_list[j] * sx_list[j + 1] + \
               Jy_list[j] * sy_list[j] * sy_list[j + 1] + \
               Jz_list[j] * sz_list[j] * sz_list[j + 1]
        
        # add H_ij to H_N and append H_ij to H_list
        H_N += H_ij
        H_list.append(H_ij)

    # execute if periodic boundary conditions are specified
    if periodic_bc: 
        
        # find H_N1, the Hamiltonian between the Nth and first site
        H_N1 = Jx_list[N-1] * sx_list[N - 1] * sx_list[0] + \
               Jy_list[N-1] * sy_list[N - 1] * sy_list[0] + \
               Jz_list[N-1] * sz_list[N - 1] * sz_list[0]

        # add H_N1 to H_N and append H_N1 to H_list
        H_N += H_N1
        H_list.append(H_N1)

    # find eigenavlues and eigenstates of Hamiltonian 
    eigenvalues, eigenstates = H_N.eigenstates()

    # find indices of smallest eigenvalues, which correspond to the energy(ies) 
    # of the ground state (space in the case of degeneracy); 
    min_eigenvalue = min(eigenvalues)
    indices = [index for index, value in enumerate(eigenvalues) if np.allclose(value, min_eigenvalue, tolerance)]

    # find eigenstates corresponding to ground state
    eigenstates_list = eigenstates[indices]

    # create sum of density matrices of ground states in case ground state is degenerate
    ρ_ground_state = 0 
    for j in range(len(eigenstates_list)):
        ρ_ground_state += (eigenstates_list[j])*(eigenstates_list[j]).dag()

    # return normalized ground state
    return H_N, H_list, eigenstates, eigenvalues, min_eigenvalue, ρ_ground_state
def homogenize_eigenstates(eigenstate_list):
    
    # convert eigenstate quantum objects to arrays
    eigenstate_arrays = np.around([eigenstate.full() for eigenstate in eigenstate_list], decimals=12)
    
    # if the first nonzero entry is negative in an array, multiply all nonzero
    # entries by -1
    for eigenstate in eigenstate_arrays: 
        if eigenstate[np.nonzero(eigenstate)[0][0]] < 0:
            nonzero_mask = eigenstate != 0 
            eigenstate[nonzero_mask] = -eigenstate[nonzero_mask]
            
    # sorted list of arrays and convert into quantum objects      
    sorted_tuples = sorted(eigenstate_arrays, key=lambda x: tuple(x))
    sorted_eigenstates = [Qobj(state) for state in sorted_tuples]
    
    return sorted_eigenstates

# $\text{N = 2}$

## $\hat{H}_2 = X_1 X_2 + Y_1 Y_2 + \lambda Z_1 Z_2 $

In [3]:
# specify parameters of 2-site spin chain
N = 2
Jx = 1
Jy = 1
Jz = 1
periodic_bc = False
tolerance = 1e-12

# calculate spin chain quantities
# start_time = time.time()
H2, H2_list, eigenstates2, eigenvalues2, min_eigenvalue2, ρ_ground_state2 = quantities(N, Jx, Jy, Jz, tolerance)
# print("%s seconds" % (time.time() - start_time))

## $\text{Matrix Product States for } \hat{H}_2 $

In [4]:
# define singlet and triplet states
S = eigenstates2[0]
T0 = eigenstates2[3]
Tn1 = eigenstates2[2]
T1 = eigenstates2[1]

# create basis of singlets and triplets
ST_basis = [S, Tn1, T0, T1]

# create matrix with S, Tn1, T0 and T1 as column vectors 
ST_matrix = Qobj(np.hstack([eigenstate.full() for eigenstate in ST_basis]))
display(ST_matrix)

# eigenvalues for 2-site spin chain
display(eigenvalues2)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.          0.          0.          1.        ]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          1.          0.          0.        ]]

array([-3.,  1.,  1.,  1.])

# $\text{N=4}$

## $\hat{H}_4 = \sum_{j = 1}^{4} X_j X_{j+1} + Y_j Y_{j+1} + \lambda Z_j Z_{j+1} $

In [5]:
# specify parameters of spin chain 
N = 4
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# calculate spin chain quantities
start_time = time.time()
H4, H4_list, eigenstates4, eigenvalues4, min_eigenvalue4, ρ_ground_state4 = quantities(N, Jx, Jy, Jz, tolerance)
# print("%s seconds" % (time.time() - start_time))

In [6]:
# collect the lists of Hamiltonians for blue and red bonds  
H_blue_list4 = H4_list[::2]
H_red_list4 = H4_list[1::2]

# diagonlize blue and red bonds so as to find the eigenvectors and associated eigenstates 
# for each blue and red bonds
blue_eigvals_array4, blue_eigvecs_list4 = simdiag(H_blue_list4)
red_eigvals_array4, red_eigvecs_list4 = simdiag(H_red_list4)

# calculate lists of blue and red Hamiltonians using spectral decomposition
H_blue_reconstructed4 = [sum([λ*vec*vec.dag() for λ, vec in zip(blue_eigvals_array4[i], blue_eigvecs_list4)]) \
                        for i in range(int(N/2))]
H_red_reconstructed4 = [sum([λ*vec*vec.dag() for λ, vec in zip(red_eigvals_array4[i], red_eigvecs_list4)]) \
                       for i in range(int(N/2))]

# ensure we can reconstructed H_blue and H_red through spectral decomposition
H_tot_bool4 = []
for Hb1, Hb2, Hr1, Hr2 in zip(H_blue_list4, H_blue_reconstructed4, H_red_list4, H_red_reconstructed4):
    H_tot_bool4.append(Hb1 == Hb2)
    H_tot_bool4.append(Hr1 == Hr2)
display(all(H_tot_bool4))

display(H_blue_list4 == H_blue_reconstructed4)

True

True

In [7]:
# collect all eigenvalue lists so that eigvals_array[j] contains the lists of eigenvalues
# for the jth spin operator
eigvals_array4 = np.concatenate((blue_eigvals_array4, red_eigvals_array4))
eigvals_array4[::2] = blue_eigvals_array4
eigvals_array4[1::2] = red_eigvals_array4

# verify that we have successfully interleaved the eigenvalues 
# of blue and red bonds to construct eigvals_array
eigvals_tot_bool4 = []
for eigval_b1, eigval_b2, eigval_r1, eigval_r2 in zip(eigvals_array4[::2], blue_eigvals_array4, \
                                                      eigvals_array4[1::2], red_eigvals_array4):
    eigvals_tot_bool4.append(eigval_b1 == eigval_b2)
    eigvals_tot_bool4.append(eigval_r1 == eigval_r2)

all(np.concatenate(eigvals_tot_bool4))

True

## $ \text{Matrix Product States for } \hat{H}_4$

In [8]:
# define local Hamiltonians generated from custom function 
H12 = H4_list[0]
H23 = H4_list[1]
H34 = H4_list[2]
H41 = H4_list[3]

# define local Hamiltonians that are constructed manually
H12_constructed = tensor(σ_x, σ_x, I2, I2) + tensor(σ_y, σ_y, I2, I2) + tensor(σ_z, σ_z, I2, I2)
H23_constructed = tensor(I2, σ_x, σ_x, I2) + tensor(I2, σ_y, σ_y, I2) + tensor(I2, σ_z, σ_z, I2)
H34_constructed = tensor(I2, I2, σ_x, σ_x) + tensor(I2, I2, σ_y, σ_y) + tensor(I2, I2, σ_z, σ_z)
H41_constructed = tensor(σ_x, I2, I2, σ_x) + tensor(σ_y, I2, I2, σ_y) + tensor(σ_z, I2, I2, σ_z)

# define operator that swaps the qubits with indices 2 and 4 for a four qubit state, i.e.
# SWAP_24|abcd> = |adcb>
H24 = tensor(I2, σ_x, I2, σ_x) + tensor(I2, σ_y, I2, σ_y) + tensor(I2, σ_z, I2, σ_z)
display(H24 == tensor([I2, σ_x]*2) + tensor([I2, σ_y]*2) + tensor([I2, σ_z]*2))
SWAP_24_composite = Qobj((H24 + I16)/2)
SWAP_24 = Qobj((H24 + I16)/2, dims = [[16], [16]])

# verify that generated and constructed Hamiltonians are equivalenet
H_constructed = [H12_constructed, H23_constructed, H34_constructed, H41_constructed]
H4_list == H_constructed

True

True

In [9]:
# compute matrix product state for blue bonds 
blue_mps_basis4 = [Qobj(eigvec) for eigvec in np.kron(ST_basis, ST_basis)]
red_mps_basis4 = [SWAP_24(Qobj(eigvec)) for eigvec in np.kron(ST_basis, ST_basis)]

# show equality between eigenstates generated by simdiag and matrix product state
blue_eigvecs_sorted4 = homogenize_eigenstates(blue_eigvecs_list4)
blue_mps_sorted4 = homogenize_eigenstates(blue_mps_basis4)
display(blue_eigvecs_sorted4 == blue_mps_sorted4)

# show equality between eigenstates generated by simdiag and matrix product state
red_eigvecs_sorted4 = homogenize_eigenstates(red_eigvecs_list4)
red_mps_sorted4 = homogenize_eigenstates(red_mps_basis4)
display(red_eigvecs_sorted4 == red_mps_sorted4)

True

True

In [10]:
# alternative way to compute the Kronecker product for two lists of the singlet-triplet basis vectors
N = 4
np.array_equal(np.kron(ST_basis, ST_basis), reduce(np.kron, [ST_basis]*(int(N/2))))

True

In [11]:
# transformation from one local Hamiltonian to another 
display(SWAP_24_composite.dag()*H41*SWAP_24_composite == H12)
display(SWAP_24_composite.dag()*H23*SWAP_24_composite == H34)

True

True

## $\text{Ordered list of spectrum for local Hamiltonians of } \hat{H}_4$

In [12]:
# specify number of sites and index of the Hamiltonian 
N = 4
index = 1 

# compute spectrum from tensor products of index 1 to i 
H12_spectrum_1i = np.tile(eigenvalues2, 2**(index -1))
#display(H12_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H12_spectrum_1N = np.repeat(H12_spectrum_1i, np.ceil(2**(N - index - 1)))
print('Eigenvalues:' + str(H12_spectrum_1N))

# compute H12 from matrix product state basis
tolerance = 1e-12
H12_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, blue_mps_basis4)])
np.allclose(H12, H12_mps, atol= tolerance)

Eigenvalues:[-3. -3. -3. -3.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


True

In [13]:
# specify index of the Hamiltonian 
index = 3 

# compute spectrum from tensor products of index 1 to i 
H34_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H34_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H34_spectrum_1N = np.repeat(H34_spectrum_1i, np.ceil(2**(N - index - 1)))
display(H34_spectrum_1N)

tolerance = 1e-12
H34_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, blue_mps_basis4)])
np.allclose(H34, H34_mps, atol=tolerance)

array([-3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,
        1.,  1.,  1.])

True

In [14]:
# specify index of the Hamiltonian 
index = 2 

# compute H23 from matrix product state basis
tolerance = 1e-12
H23_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, red_mps_basis4)])
np.allclose(H23, H23_mps, atol= tolerance)

True

In [15]:
# specify index of the Hamiltonian 
index = 4

# compute H41 from matrix product state basis
tolerance = 1e-12
H41_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, red_mps_basis4)])
np.allclose(H41, H41_mps, atol= tolerance)

True

# $\text{N=6}$

## $\hat{H}_6 = \sum_{j = 1}^{6} X_j X_{j+1} + Y_j Y_{j+1} + \lambda Z_j Z_{j+1} $

In [16]:
# specify parameters of spin chain 
N = 6
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# calculate spin chain quantities
start_time = time.time()
H6, H6_list, eigenstates6, eigenvalues6, min_eigenvalue6, ρ_ground_state6 = quantities(N, Jx, Jy, Jz, tolerance)
# print("%s seconds" % (time.time() - start_time))

In [17]:
# collect the lists of Hamiltonians for blue and red bonds  
H_blue_list6 = H6_list[::2]
H_red_list6 = H6_list[1::2]

# diagonlize blue and red bonds so as to find the eigenvectors and associated eigenstates 
# for each blue and red bonds
blue_eigvals_array6, blue_eigvecs_list6 = simdiag(H_blue_list6)
red_eigvals_array6, red_eigvecs_list6 = simdiag(H_red_list6)

# calculate lists of blue and red Hamiltonians using spectral decomposition
H_blue_reconstructed6 = [sum([λ*vec*vec.dag() for λ, vec in zip(blue_eigvals_array6[i], blue_eigvecs_list6)]) \
                        for i in range(int(N/2))]
H_red_reconstructed6 = [sum([λ*vec*vec.dag() for λ, vec in zip(red_eigvals_array6[i], red_eigvecs_list6)]) \
                       for i in range(int(N/2))]

# ensure we can reconstructed H_blue and H_red through spectral decomposition
H_tot_bool6 = []
for Hb1, Hb2, Hr1, Hr2 in zip(H_blue_list6, H_blue_reconstructed6, H_red_list6, H_red_reconstructed6):
    H_tot_bool6.append(Hb1 == Hb2)
    H_tot_bool6.append(Hr1 == Hr2)
all(H_tot_bool6)

True

In [18]:
# collect all eigenvalue lists so that eigvals_array[j] contains the lists of eigenvalues
# for the jth spin operator
eigvals_array6 = np.concatenate((blue_eigvals_array6, red_eigvals_array6))
eigvals_array6[::2] = blue_eigvals_array6
eigvals_array6[1::2] = red_eigvals_array6

# verify that we have successfully interleaved the eigenvalues 
# of blue and red bonds to construct eigvals_array
eigvals_tot_bool6 = []
for eigval_b1, eigval_b2, eigval_r1, eigval_r2 in zip(eigvals_array6[::2], blue_eigvals_array6, \
                                                      eigvals_array6[1::2], red_eigvals_array6):
    eigvals_tot_bool6.append(eigval_b1 == eigval_b2)
    eigvals_tot_bool6.append(eigval_r1 == eigval_r2)

all(np.concatenate(eigvals_tot_bool6))

True

## $ \text{Matrix Product States for } \hat{H}_6$

In [19]:
N = 6
blue_mps_basis6 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [20]:
blue_mps_sorted6 = homogenize_eigenstates(blue_mps_basis6)
blue_eigvecs_sorted6 = homogenize_eigenstates(blue_eigvecs_list6)
display(blue_eigvecs_sorted6 == blue_mps_sorted6)

True

In [21]:
# define H26
H26 = tensor(I2, σ_x, I2, I2, I2, σ_x) + \
      tensor(I2, σ_y, I2, I2, I2, σ_y) + \
      tensor(I2, σ_z, I2, I2, I2, σ_z)

# define number of sites and indices where we desire σ_x
N = 6
site1 = 2
site2 = 6

# define list of identity operators and where we desire Pauli operators
op_list = [qeye(2)]*N
op_list[site1 - 1] = σ_x
op_list[site2 - 1] = σ_x
HX26 = tensor(op_list)
op_list[site1 - 1] = σ_y
op_list[site2 - 1] = σ_y
HY26 = tensor(op_list)
op_list[site1 - 1] = σ_z
op_list[site2 - 1] = σ_z
HZ26 = tensor(op_list) 
H26_constructed = HX26 + HY26 + HZ26
H26 == H26_constructed

True

In [22]:
# define H35
H35 = tensor(I2, I2, σ_x, I2, σ_x, I2) + \
      tensor(I2, I2, σ_y, I2, σ_y, I2) + \
      tensor(I2, I2, σ_z, I2, σ_z, I2)

# define number of sites and indices where we desire Pauli operators
N = 6
site1 = 3
site2 = 5

# define list of identity operators and where you want our σ_x to be
op_list = [qeye(2)]*N
op_list[site1 - 1] = σ_x
op_list[site2 - 1] = σ_x
HX35 = tensor(op_list)
op_list[site1 - 1] = σ_y
op_list[site2 - 1] = σ_y
HY35 = tensor(op_list)
op_list[site1 - 1] = σ_z
op_list[site2 - 1] = σ_z
HZ35 = tensor(op_list)
H35_constructed = HX35 + HY35 + HZ35
H35 == H35_constructed

True

In [23]:
# define operator that swaps sites 2 and 6 as well as 3 and 5
I64 = tensor([qeye(2)]*6)
SWAP_26_35 = Qobj(((H26 + I64)/2)*(H35 + I64)/2, dims = [[64], [64]])

In [24]:
# compute matrix product state for blue bonds 
N = 6
blue_mps_basis6 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
red_mps_basis6 = [SWAP_26_35(Qobj(eigvec)) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [25]:
blue_mps_sorted6 = homogenize_eigenstates(blue_mps_basis6)
blue_eigvecs_sorted6 = homogenize_eigenstates(blue_eigvecs_list6)
display(blue_eigvecs_sorted6 == blue_mps_sorted6)

True

In [26]:
red_mps_sorted6 = homogenize_eigenstates(red_mps_basis6)
red_eigvecs_sorted6 = homogenize_eigenstates(red_eigvecs_list6)
display(red_eigvecs_sorted6 == red_mps_sorted6)

True

## $\text{Ordered list of eigenspectrum for } \hat{H}_6$

In [27]:
# specify number of sites and index of the Hamiltonian 
N = 6
index = 1 

# compute spectrum from tensor products of index 1 to i 
H12_spectrum_1i = np.tile(eigenvalues2, 2**(index -1))
#display(H12_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H12_spectrum_1N = np.repeat(H12_spectrum_1i, np.ceil(2**(N - index - 1)))
print('Eigenvalues:' + str(H12_spectrum_1N))

# compute H12 from matrix product state basis
tolerance = 1e-12
H12_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, blue_mps_basis6)])
np.allclose(H6_list[0], H12_mps, atol= tolerance)

Eigenvalues:[-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


True

In [28]:
# specify index of the Hamiltonian 
index = 3 

# compute spectrum from tensor products of index 1 to i 
H34_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H34_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H34_spectrum_1N = np.repeat(H34_spectrum_1i, np.ceil(2**(N - index - 1)))
display(H34_spectrum_1N)

tolerance = 1e-12
H34_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, blue_mps_basis6)])
np.allclose(H6_list[2], H34_mps, atol=tolerance)

array([-3., -3., -3., -3.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -3., -3., -3., -3.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., -3., -3., -3., -3.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -3., -3., -3., -3.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

True

In [29]:
# specify index of the Hamiltonian 
index = 5

# compute spectrum from tensor products of index 1 to i 
H56_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H56_spectrum_1N = np.repeat(H56_spectrum_1i, np.ceil(2**(N - index - 1)))
display(H56_spectrum_1N)

tolerance = 1e-12
H56_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, blue_mps_basis6)])
np.allclose(H6_list[4], H56_mps, atol=tolerance)

array([-3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,
        1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,
        1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,
        1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1.,
       -3.,  1.,  1.,  1., -3.,  1.,  1.,  1., -3.,  1.,  1.,  1.])

True

In [30]:
# specify index of the Hamiltonian 
index = 2 

# compute H23 from matrix product state basis
tolerance = 1e-12
H23_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, red_mps_basis6)])
np.allclose(H6_list[1], H23_mps, atol= tolerance)

True

In [31]:
# specify index of the Hamiltonian 
index = 4

# compute H41 from matrix product state basis
tolerance = 1e-12
H45_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, red_mps_basis6)])
np.allclose(H6_list[3], H45_mps, atol= tolerance)

True

In [32]:
# specify index of the Hamiltonian 
index = 6

# compute H41 from matrix product state basis
tolerance = 1e-12
H61_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, red_mps_basis6)])
np.allclose(H6_list[5], H61_mps, atol= tolerance)

True

# $ N = 8 $

## $\hat{H}_8 = \sum_{j = 1}^{8} X_j X_{j+1} + Y_j Y_{j+1} + \lambda Z_j Z_{j+1} $

In [33]:
# specify parameters of spin chain 
N = 8
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# calculate spin chain quantities
start_time = time.time()
H8, H8_list, eigenstates8, eigenvalues8, min_eigenvalue8, ρ_ground_state8 = quantities(N, Jx, Jy, Jz, tolerance)
# print("%s seconds" % (time.time() - start_time))

In [34]:
# collect the lists of Hamiltonians for blue and red bonds  
H_blue_list8 = H8_list[::2]
H_red_list8 = H8_list[1::2]

# diagonlize blue and red bonds so as to find the eigenvectors and associated eigenstates 
# for each blue and red bonds
blue_eigvals_array8, blue_eigvecs_list8 = simdiag(H_blue_list8)
red_eigvals_array8, red_eigvecs_list8 = simdiag(H_red_list8)

# calculate lists of blue and red Hamiltonians using spectral decomposition
H_blue_reconstructed8 = [sum([λ*vec*vec.dag() for λ, vec in zip(blue_eigvals_array8[i], blue_eigvecs_list8)]) \
                        for i in range(int(N/2))]
H_red_reconstructed8 = [sum([λ*vec*vec.dag() for λ, vec in zip(red_eigvals_array8[i], red_eigvecs_list8)]) \
                       for i in range(int(N/2))]

# ensure we can reconstructed H_blue and H_red through spectral decomposition
H_tot_bool8 = []
for Hb1, Hb2, Hr1, Hr2 in zip(H_blue_list8, H_blue_reconstructed8, H_red_list8, H_red_reconstructed8):
    H_tot_bool8.append(Hb1 == Hb2)
    H_tot_bool8.append(Hr1 == Hr2)
all(H_tot_bool8)

True

In [35]:
# collect all eigenvalue lists so that eigvals_array[j] contains the lists of eigenvalues
# for the jth spin operator
eigvals_array8 = np.concatenate((blue_eigvals_array8, red_eigvals_array8))
eigvals_array8[::2] = blue_eigvals_array8
eigvals_array8[1::2] = red_eigvals_array8

# verify that we have successfully interleaved the eigenvalues 
# of blue and red bonds to construct eigvals_array
eigvals_tot_bool8 = []
for eigval_b1, eigval_b2, eigval_r1, eigval_r2 in zip(eigvals_array8[::2], blue_eigvals_array8, \
                                                      eigvals_array8[1::2], red_eigvals_array8):
    eigvals_tot_bool8.append(eigval_b1 == eigval_b2)
    eigvals_tot_bool8.append(eigval_r1 == eigval_r2)

all(np.concatenate(eigvals_tot_bool8))

True

## $ \text{Matrix Product States for } \hat{H}_8 $

In [36]:
N = 8
blue_mps_basis8 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [37]:
blue_mps_sorted8 = homogenize_eigenstates(blue_mps_basis8)
blue_eigvecs_sorted8 = homogenize_eigenstates(blue_eigvecs_list8)
display(blue_eigvecs_sorted8 == blue_mps_sorted8)

True

In [38]:
# define H28
H28 = tensor(I2, σ_x, I2, I2, I2, I2, I2, σ_x) + \
      tensor(I2, σ_y, I2, I2, I2, I2, I2, σ_y) + \
      tensor(I2, σ_z, I2, I2, I2, I2, I2, σ_z) 

# define number of sites and indices where we desire σ_x
N = 8
site1 = 2
site2 = 8

# define list of identity operators and where we desire Pauli operators
op_list = [qeye(2)]*N
op_list[site1 - 1] = σ_x
op_list[site2 - 1] = σ_x
HX28 = tensor(op_list)
op_list[site1 - 1] = σ_y
op_list[site2 - 1] = σ_y
HY28 = tensor(op_list)
op_list[site1 - 1] = σ_z
op_list[site2 - 1] = σ_z
HZ28 = tensor(op_list) 
H28_constructed = HX28 + HY28 + HZ28
H28 == H28_constructed

True

In [39]:
# define H37
H37 = tensor(I2, I2, σ_x, I2, I2, I2, σ_x, I2) + \
      tensor(I2, I2, σ_y, I2, I2, I2, σ_y, I2) + \
      tensor(I2, I2, σ_z, I2, I2, I2, σ_z, I2)

# define number of sites and indices where we desire Pauli operators
N = 8
site1 = 3
site2 = 7

# define list of identity operators and where you want our σ_x to be
op_list = [qeye(2)]*N
op_list[site1 - 1] = σ_x
op_list[site2 - 1] = σ_x
HX37 = tensor(op_list)
op_list[site1 - 1] = σ_y
op_list[site2 - 1] = σ_y
HY37 = tensor(op_list)
op_list[site1 - 1] = σ_z
op_list[site2 - 1] = σ_z
HZ37 = tensor(op_list)
H37_constructed = HX37 + HY37 + HZ37
H37 == H37_constructed

True

In [40]:
# define H46
H46 = tensor(I2, I2, I2, σ_x, I2, σ_x, I2, I2) + \
      tensor(I2, I2, I2, σ_y, I2, σ_y, I2, I2) + \
      tensor(I2, I2, I2, σ_z, I2, σ_z, I2, I2)

# define number of sites and indices where we desire Pauli operators
N = 8
site1 = 4
site2 = 6

# define list of identity operators and where you want our σ_x to be
op_list = [qeye(2)]*N
op_list[site1 - 1] = σ_x
op_list[site2 - 1] = σ_x
HX46 = tensor(op_list)
op_list[site1 - 1] = σ_y
op_list[site2 - 1] = σ_y
HY46 = tensor(op_list)
op_list[site1 - 1] = σ_z
op_list[site2 - 1] = σ_z
HZ46 = tensor(op_list)
H46_constructed = HX46 + HY46 + HZ46
H46 == H46_constructed

True

In [41]:
# define operator that swaps sites 2 and 6 as well as 3 and 7
I256 = tensor([qeye(2)]*N)
SWAP_28_37_46 = Qobj(((H28 + I256)/2)*((H37 + I256)/2)*((H46 + I256)/2), dims = [[256], [256]])

In [42]:
# compute matrix product state for blue bonds 
N = 8
blue_mps_basis8 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
red_mps_basis8 = [SWAP_28_37_46(Qobj(eigvec)) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [43]:
blue_mps_sorted8 = homogenize_eigenstates(blue_mps_basis8)
blue_eigvecs_sorted8 = homogenize_eigenstates(blue_eigvecs_list8)
display(blue_eigvecs_sorted8 == blue_mps_sorted8)

True

In [44]:
red_mps_sorted6 = homogenize_eigenstates(red_mps_basis6)
red_eigvecs_sorted6 = homogenize_eigenstates(red_eigvecs_list6)
display(red_eigvecs_sorted6 == red_mps_sorted6)

True

## $\text{Ordered list of eigenspectrum for } \hat{H}_8 $

In [45]:
# specify number of sites and index of the Hamiltonian 
N = 8
index = 1 

# compute spectrum from tensor products of index 1 to i 
H12_spectrum_1i = np.tile(eigenvalues2, 2**(index -1))
#display(H12_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H12_spectrum_1N = np.repeat(H12_spectrum_1i, np.ceil(2**(N - index - 1)))
# print('Eigenvalues:' + str(H12_spectrum_1N))

# compute H12 from matrix product state basis
tolerance = 1e-12
H12_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, blue_mps_basis8)])
np.allclose(H8_list[0], H12_mps, atol= tolerance)

True

In [46]:
# specify index of the Hamiltonian 
index = 3 

# compute spectrum from tensor products of index 1 to i 
H34_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H34_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H34_spectrum_1N = np.repeat(H34_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H34_spectrum_1N)

tolerance = 1e-12
H34_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, blue_mps_basis8)])
np.allclose(H8_list[2], H34_mps, atol=tolerance)

True

In [47]:
# specify index of the Hamiltonian 
index = 5

# compute spectrum from tensor products of index 1 to i 
H56_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H56_spectrum_1N = np.repeat(H56_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H56_spectrum_1N)

tolerance = 1e-12
H56_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, blue_mps_basis8)])
np.allclose(H8_list[4], H56_mps, atol=tolerance)

True

In [48]:
# specify index of the Hamiltonian 
index = 7

# compute spectrum from tensor products of index 1 to i 
H78_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H78_spectrum_1N = np.repeat(H78_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H56_spectrum_1N)

tolerance = 1e-12
H78_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H78_spectrum_1N, blue_mps_basis8)])
np.allclose(H8_list[6], H78_mps, atol=tolerance)

True

In [49]:
# specify index of the Hamiltonian 
index = 2 

# compute H23 from matrix product state basis
tolerance = 1e-12
H23_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H78_spectrum_1N, red_mps_basis8)])
np.allclose(H8_list[1], H23_mps, atol= tolerance)

True

In [50]:
# specify index of the Hamiltonian 
index = 4

# compute H41 from matrix product state basis
tolerance = 1e-12
H45_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, red_mps_basis8)])
np.allclose(H8_list[3], H45_mps, atol= tolerance)

True

In [51]:
# specify index of the Hamiltonian 
index = 6

# compute H41 from matrix product state basis
tolerance = 1e-12
H67_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, red_mps_basis6)])
np.allclose(H6_list[5], H67_mps, atol= tolerance)

True

In [52]:
# specify index of the Hamiltonian 
index = 8

# compute H41 from matrix product state basis
tolerance = 1e-12
H81_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, red_mps_basis8)])
np.allclose(H8_list[7], H81_mps, atol= tolerance)

True

# $ N = 10$

## $\hat{H}_{10} = \sum_{j = 1}^{10} X_j X_{j+1} + Y_j Y_{j+1} + \lambda Z_j Z_{j+1} $

In [56]:
# specify parameters of spin chain 
N = 10
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# calculate spin chain quantities
start_time = time.time()
H10, H10_list, eigenstates10, eigenvalues10, min_eigenvalue10, ρ_ground_state10 = quantities(N, Jx, Jy, Jz, tolerance)
# print("%s seconds" % (time.time() - start_time))

In [57]:
# collect the lists of Hamiltonians for blue and red bonds  
H_blue_list10 = H10_list[::2]
H_red_list10 = H10_list[1::2]

# diagonlize blue and red bonds so as to find the eigenvectors and associated eigenstates 
# for each blue and red bonds
blue_eigvals_array10, blue_eigvecs_list10 = simdiag(H_blue_list10)
red_eigvals_array10, red_eigvecs_list10 = simdiag(H_red_list10)

# calculate lists of blue and red Hamiltonians using spectral decomposition
H_blue_reconstructed10 = [sum([λ*vec*vec.dag() for λ, vec in zip(blue_eigvals_array10[i], blue_eigvecs_list10)]) \
                        for i in range(int(N/2))]
H_red_reconstructed10 = [sum([λ*vec*vec.dag() for λ, vec in zip(red_eigvals_array10[i], red_eigvecs_list10)]) \
                       for i in range(int(N/2))]

# ensure we can reconstructed H_blue and H_red through spectral decomposition
H_tot_bool10 = []
for Hb1, Hb2, Hr1, Hr2 in zip(H_blue_list10, H_blue_reconstructed10, H_red_list10, H_red_reconstructed10):
    H_tot_bool10.append(Hb1 == Hb2)
    H_tot_bool10.append(Hr1 == Hr2)
all(H_tot_bool10)

True

In [58]:
# collect all eigenvalue lists so that eigvals_array[j] contains the lists of eigenvalues
# for the jth spin operator
eigvals_array10 = np.concatenate((blue_eigvals_array10, red_eigvals_array10))
eigvals_array10[::2] = blue_eigvals_array10
eigvals_array10[1::2] = red_eigvals_array10

# verify that we have successfully interleaved the eigenvalues 
# of blue and red bonds to construct eigvals_array
eigvals_tot_bool10 = []
for eigval_b1, eigval_b2, eigval_r1, eigval_r2 in zip(eigvals_array10[::2], blue_eigvals_array10, \
                                                      eigvals_array10[1::2], red_eigvals_array10):
    eigvals_tot_bool10.append(eigval_b1 == eigval_b2)
    eigvals_tot_bool10.append(eigval_r1 == eigval_r2)

all(np.concatenate(eigvals_tot_bool10))

True

## $ \text{Matrix Product States for } \hat{H}_{10} $

In [59]:
N = 10
blue_mps_basis10 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [60]:
blue_mps_sorted10 = homogenize_eigenstates(blue_mps_basis10)
blue_eigvecs_sorted10 = homogenize_eigenstates(blue_eigvecs_list10)
display(blue_eigvecs_sorted10 == blue_mps_sorted10)

True

In [61]:
N = 10
H_SWAP_indices = [(2 + i, N - i) for i in range(0, int(N/2) - 1)]
H_SWAP_indices

[(2, 10), (3, 9), (4, 8), (5, 7)]

In [62]:
# initialize empty list to fill with H_SWAP matrices, which are Hamiltonians containing 
# XX + YY + ZZ at indices specified by each tuple in H_SWAP_indices
H_SWAP_list = []

for indices_tuple in H_SWAP_indices:
    
    # collect at which sites we will insert Pauli matrices
    site1 = indices_tuple[0]
    site2 = indices_tuple[1]
    
    # define list of identity operators and where we desire Pauli operators
    op_list = [qeye(2)]*N
    op_list[site1 - 1] = σ_x
    op_list[site2 - 1] = σ_x
    HX_couplings = tensor(op_list)
    op_list[site1 - 1] = σ_y
    op_list[site2 - 1] = σ_y
    HY_couplings = tensor(op_list)
    op_list[site1 - 1] = σ_z
    op_list[site2 - 1] = σ_z
    HZ_couplings = tensor(op_list) 
    
    # append H_SWAP = HX_couplings + HY_couplings + HZ_couplings to H_SWAP_mat_list
    H_SWAP_list.append(HX_couplings + HY_couplings + HZ_couplings)
    

In [63]:
# define operator that swaps sites 2 and 6 as well as 3 and 7
I = tensor([qeye(2)]*N)

In [64]:
# specify the dimension of the new matrix 
dim = 2**N
H_SWAP_tot = Qobj(reduce(lambda x,y: x*y , [(H_SWAP + I)/2 for H_SWAP in H_SWAP_list]), dims = [[dim], [dim]])

In [65]:
# compute matrix product state for blue bonds 
N = 10
blue_mps_basis10 = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
red_mps_basis10 = [H_SWAP_tot(Qobj(eigvec)) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]

In [66]:
blue_mps_sorted10 = homogenize_eigenstates(blue_mps_basis10)
blue_eigvecs_sorted10 = homogenize_eigenstates(blue_eigvecs_list10)
display(blue_eigvecs_sorted10 == blue_mps_sorted10)

True

In [67]:
red_mps_sorted10 = homogenize_eigenstates(red_mps_basis10)
red_eigvecs_sorted10 = homogenize_eigenstates(red_eigvecs_list10)
display(red_eigvecs_sorted10 == red_mps_sorted10)

True

## $ \text{Ordered list of eigenspectrum for } \hat{H}_{10} $

In [68]:
# specify number of sites and index of the Hamiltonian 
N = 10
index = 1 

# compute spectrum from tensor products of index 1 to i 
H12_spectrum_1i = np.tile(eigenvalues2, 2**(index -1))
#display(H12_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H12_spectrum_1N = np.repeat(H12_spectrum_1i, np.ceil(2**(N - index - 1)))
# print('Eigenvalues:' + str(H12_spectrum_1N))

# compute H12 from matrix product state basis
tolerance = 1e-12
H12_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, blue_mps_basis10)])
np.allclose(H10_list[0], H12_mps, atol= tolerance)

True

In [69]:
# specify index of the Hamiltonian 
index = 3 

# compute spectrum from tensor products of index 1 to i 
H34_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H34_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H34_spectrum_1N = np.repeat(H34_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H34_spectrum_1N)

tolerance = 1e-12
H34_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, blue_mps_basis10)])
np.allclose(H10_list[2], H34_mps, atol=tolerance)

True

In [70]:
# specify index of the Hamiltonian 
index = 5

# compute spectrum from tensor products of index 1 to i 
H56_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H56_spectrum_1N = np.repeat(H56_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H56_spectrum_1N)

tolerance = 1e-12
H56_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, blue_mps_basis10)])
np.allclose(H10_list[4], H56_mps, atol=tolerance)

True

In [71]:
# specify index of the Hamiltonian 
index = 7

# compute spectrum from tensor products of index 1 to i 
H78_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H78_spectrum_1N = np.repeat(H78_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H56_spectrum_1N)

tolerance = 1e-12
H78_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H78_spectrum_1N, blue_mps_basis10)])
np.allclose(H10_list[6], H78_mps, atol=tolerance)

True

In [72]:
# specify index of the Hamiltonian 
index = 9

# compute spectrum from tensor products of index 1 to i 
H78_spectrum_1i = np.tile(eigenvalues2, 2**(index-1))
# display(H56_spectrum_1i)

# compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
H9_10_spectrum_1N = np.repeat(H78_spectrum_1i, np.ceil(2**(N - index - 1)))
# display(H56_spectrum_1N)

tolerance = 1e-12
H9_10_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H9_10_spectrum_1N, blue_mps_basis10)])
np.allclose(H10_list[8], H9_10_mps, atol=tolerance)

True

In [73]:
# specify index of the Hamiltonian 
index = 2 

# compute H23 from matrix product state basis
tolerance = 1e-12
H23_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H9_10_spectrum_1N, red_mps_basis10)])
np.allclose(H10_list[1], H23_mps, atol= tolerance)

True

In [74]:
# specify index of the Hamiltonian 
index = 4

# compute H41 from matrix product state basis
tolerance = 1e-12
H45_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H78_spectrum_1N, red_mps_basis10)])
np.allclose(H10_list[3], H45_mps, atol= tolerance)

True

In [75]:
# specify index of the Hamiltonian 
index = 6

# compute H41 from matrix product state basis
tolerance = 1e-12
H67_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H56_spectrum_1N, red_mps_basis10)])
np.allclose(H10_list[5], H67_mps, atol= tolerance)

True

In [76]:
# specify index of the Hamiltonian 
index = 8

# compute H41 from matrix product state basis
tolerance = 1e-12
H81_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H34_spectrum_1N, red_mps_basis10)])
np.allclose(H10_list[7], H81_mps, atol= tolerance)

True

In [77]:
# specify index of the Hamiltonian 
index = 10

# compute H41 from matrix product state basis
tolerance = 1e-12
H10_1_mps = sum([λ*vec*vec.dag() for λ, vec in zip(H12_spectrum_1N, red_mps_basis10)])
np.allclose(H10_list[9], H10_1_mps, atol= tolerance)

True

# $ N = 12 $

In [92]:
# specify parameters of spin chain 
N = 12
Jx = 1
Jy = 1
Jz = 1
periodic_bc = True
tolerance = 1e-12

# calculate spin chain quantities
start_time = time.time()
H12, H12_list, eigenstates12, eigenvalues12, min_eigenvalue12, ρ_ground_state12 = \
quantities(N, Jx, Jy, Jz, tolerance)
print("%s seconds" % (time.time() - start_time))

49.99112391471863 seconds


# Global Eigendecomposition 

In [53]:
def eigendecomp_spin_operators(N, ST_basis):
    
    '''
    function to find the global eigenkets for blue and red bonds as well
    as the spectrum of each local 2-site Hamiltonian in the basis of these
    global eigenkets
    '''
    
    # collect indices for H_ik Hamiltonians which will be used to convert 
    # blue global eigenkets to red global eigenkets
    H_ik_indices = [(2 + i, N - i) for i in range(0, int(N/2) - 1)]

    # collect all H_ik matrices, which are Hamiltonians containing 
    # XX + YY + ZZ at indices i and k specified by each tuple in H_SWAP_indices
    H_ik_list = []
    for indices_tuple in H_ik_indices:

        # collect at which sites we will insert Pauli matrices
        site1 = indices_tuple[0]
        site2 = indices_tuple[1]

        # define list of identity operators and insert Pauli matrices
        # at sites specified by indices_tuple
        op_list = [qeye(2)]*N
        op_list[site1 - 1] = σ_x
        op_list[site2 - 1] = σ_x
        HX_couplings = tensor(op_list)
        op_list[site1 - 1] = σ_y
        op_list[site2 - 1] = σ_y
        HY_couplings = tensor(op_list)
        op_list[site1 - 1] = σ_z
        op_list[site2 - 1] = σ_z
        HZ_couplings = tensor(op_list) 

        # append H_SWAP = HX_couplings + HY_couplings + HZ_couplings to H_ik_list
        H_ik_list.append(HX_couplings + HY_couplings + HZ_couplings)

    # collect product of SWAP matrices that will convert red eigenkets to blue eigenkets
    I = tensor([qeye(2)]*N)
    dim = 2**N
    SWAP_operator = Qobj(reduce(lambda x,y: x*y, \
                                [(H_ik + I)/2 for H_ik in H_ik_list]), dims = [[dim], [dim]])

    # construct global eigenkets for red and blue bonds for N sites
    blue_mpsN = [Qobj(eigvec) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
    red_mpsN = [SWAP_operator(Qobj(eigvec)) for eigvec in reduce(np.kron, [ST_basis]*(int(N/2)))]
    
    # initialize empty list which will contain the spectrum of each local Hamiltonian 
    # in the basis of the blue and red global eigenkets
    H_local_spectrum_list = []

    # find spectra of all 2-site Hamiltonians
    for i in range(1, N+1, 2):

        # compute spectrum from tensor products of index 1 to i 
        Hij_spectrum_1i = np.tile(eigenvalues2, 2**(i-1))

        # compute spectrum of tensor products of inex 1 to N, i.e., the full spectrum
        Hij_spectrum_1N = np.repeat(Hij_spectrum_1i, np.ceil(2**(N - i - 1)))

        # append to Hij_spectrum_list
        H_local_spectrum_list.append(Hij_spectrum_1N)

    # extra spectra for blue and red bonds according to previous ordering of eigenkets
    blue_spectraN = H_local_spectrum_list
    red_spectraN = H_local_spectrum_list[::-1]

    return blue_spectraN, blue_mpsN, red_spectraN, red_mpsN

In [54]:
def reconstruct_HN_list(HN_list, blue_spectraN, blue_mpsN, red_spectraN, red_mpsN):

    '''
    reconstruct list of local Hamiltonians from parameters yielded by the eigendecomposition 
    of all red and blue bonds
    '''

    # calculate the number of sites in spin chain
    N = 2*len(blue_spectraN)

    # reconstruct lists of Hamiltonians corresponding to blue and red bonds
    HN_blue_list_reconstructed = [sum([λ*vec*vec.dag() for λ, vec in \
                                  zip(blue_spectraN[i], blue_mpsN)]) for i in range(int(N/2))]
    HN_red_list_reconstructed = [sum([λ*vec*vec.dag() for λ, vec in \
                                 zip(red_spectraN[i], red_mpsN)]) for i in range(int(N/2))]

    # interleave HN_blue_list and HN_red_list to create an ordered list of local Hamiltonians 
    HN_list_reconstructed = np.concatenate((HN_blue_list_reconstructed, HN_red_list_reconstructed))
    HN_list_reconstructed[::2] = HN_blue_list_reconstructed
    HN_list_reconstructed[1::2] = HN_red_list_reconstructed 

    # extract lists of blue and red Hamiltonians from HN_list
    HN_blue_list = HN_list[::2]
    HN_red_list = HN_list[1::2]

    # verify that HN_list == HN_list_reconstructed using two methods
    verification1 = np.allclose(HN_list, HN_list_reconstructed, atol = 1e-12)

    # use for loop to check whether HN_list[i] == HN_list_reconstructed[i]
    H_tot_boolN = []
    for Hb1, Hb2, Hr1, Hr2 in zip(HN_blue_list_reconstructed, HN_blue_list, \
                                  HN_red_list_reconstructed, HN_red_list):

        H_tot_boolN.append(np.allclose(Hb1, Hb2, atol = 1e-12))
        H_tot_boolN.append(np.allclose(Hr1, Hr2, atol = 1e-12))

    verification2 = all(H_tot_boolN)

    # print whether both methods of verifying equality between Hamiltonians 
    # yield True or False
    if verification1 and verification2: 
        print('Hamiltonians are the same!')
    else: 
        print('Hamiltonians are different')

In [55]:
def construct_eig_tuples(blue_spectraN, blue_mpsN, red_spectraN, red_mpsN):
    
    '''
    construct list of tuples such that the ith entry is a tuple containing the list
    of eigenvalues and eigenvectors for the ith spin operator
    '''
    
    # combine lists of tuples into list so that the ith entry is a tuple containing the 
    # list of eigenvalues and eigenvectors for the ith local Hamiltonian
    blue_eig_tuplesN = [tuple(zip(spectra, blue_mpsN)) for spectra in blue_spectraN]
    red_eig_tuplesN = [tuple(zip(spectra, red_mpsN)) for spectra in red_spectraN]

    # interleave list for blue and red lists of tuples to create one tuple 
    eig_tuplesN = blue_eig_tuplesN + red_eig_tuplesN
    eig_tuplesN[::2] = blue_eig_tuplesN
    eig_tuplesN[1::2] = red_eig_tuplesN
    
    return eig_tuplesN

In [56]:
# verify eigendecomposition can reconstruct list of local Hamiltonians for N = 4
N = 4
blue_spectra4, blue_mps4, red_spectra4, red_mps4 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H4_list, blue_spectra4, blue_mps4, red_spectra4, red_mps4)

Hamiltonians are the same!


In [57]:
# verify eigendecomposition can reconstruct list of local Hamiltonians for N = 6
N = 6
blue_spectra6, blue_mps6, red_spectra6, red_mps6 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H6_list, blue_spectra6, blue_mps6, red_spectra6, red_mps6)

Hamiltonians are the same!


In [58]:
# verify eigendecomposition can reconstruct list of local Hamiltonians for N = 8
N = 8
blue_spectra8, blue_mps8, red_spectra8, red_mps8 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H8_list, blue_spectra8, blue_mps8, red_spectra8, red_mps8)

Hamiltonians are the same!


In [60]:
# # verify eigendecomposition can reconstruct list of local Hamiltonians for N = 10
N = 10
blue_spectra10, blue_mps10, red_spectra10, red_mps10 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H10_list, blue_spectra10, blue_mps10, red_spectra10, red_mps10)

In [93]:
# verify eigendecomposition can reconstruct list of local Hamiltonians for N = 12
N = 12
blue_spectra12, blue_mps12, red_spectra12, red_mps12 = eigendecomp_spin_operators(N, ST_basis)
reconstruct_HN_list(H12_list, blue_spectra12, blue_mps12, red_spectra12, red_mps12)

Hamiltonians are the same!


# $\text{Simulations for } N = 4$ 

In [592]:
# define local Hamiltonians 
A = H4_list[0]
B = H4_list[1]
C = H4_list[2]
D = H4_list[3]

In [593]:
# define displacement parameter
α = 1

# collect an ordered list of all the spectra of the local Hamiltonians
spectra4 = blue_spectra4 + red_spectra4
spectra4[::2] = blue_spectra4
spectra4[1::2] = red_spectra4

# define lists of eigenvalues for spin operators A, B, C and D
a = spectra4[0]
b = spectra4[1]
c = spectra4[2]
d = spectra4[3]

# define a Gaussian weight tensor
W4 = lambda α, i, j, k, l: np.exp(-((α**2)/2)*((a[i] + b[j] + c[k] + d[l])**2))

# precompute Gaussian weight tensor
W4_tensor = np.empty((len(a), len(b), len(c), len(d)))
for k3 in range(len(a)):
    for k2 in range(len(b)):
        for k1 in range(len(c)):
            for k0 in range(len(d)):
                W4_tensor[k0, k1, k2, k3] = W4(α, k0, k1, k2, k3)

# define all the bras and kets of the blue and red global eigenstates
μ4_kets = blue_mps4
μ4_bras = [eigvec.dag() for eigvec in μ4_kets]
ν4_kets = red_mps4
ν4_bras = [eigvec.dag() for eigvec in ν4_kets]

In [594]:
VTA4_1 = Qobj(np.zeros((16, 16)))
start_time = time.time()
for k3 in range(len(d)):
    for k2 in range(len(c)):
        for k1 in range(len(b)):
            for k0 in range(len(a)):
                VTA4_1 += W4(α, k0, k3, k2, k1)*W4(α, k0, k1, k2, k3) * \
                         W4(α, k2, k1, k0, k3)*W4(α, k2, k3, k0, k1) * \
                         (ν4_bras[k3]*μ4_kets[k2])*(μ4_bras[k2]*ν4_kets[k1])*(ν4_bras[k1]*μ4_kets[k0]) * \
                         ν4_kets[k3]*μ4_bras[k0]      
print("%s seconds" % (time.time() - start_time))                

14.74341607093811 seconds


In [595]:
VTA4_2 = Qobj(np.zeros((16, 16)))
start_time = time.time()
for k3 in range(len(d)):
    for k2 in range(len(c)):
        for k1 in range(len(b)):
            for k0 in range(len(a)):
                VTA4_2 += W4_tensor[k0, k3, k2, k1]*W4_tensor[k0, k1, k2, k3] * \
                         W4_tensor[k2, k1, k0, k3]*W4_tensor[k2, k3, k0, k1] * \
                         (ν4_bras[k3]*μ4_kets[k2])*(μ4_bras[k2]*ν4_kets[k1])*(ν4_bras[k1]*μ4_kets[k0]) * \
                         ν4_kets[k3]*μ4_bras[k0]      
print("%s seconds" % (time.time() - start_time))

14.304529905319214 seconds


In [596]:
start_time = time.time()
VTA4_3 = Qobj(np.zeros((16, 16)))
VTA4_3 = sum(
            W4_tensor[k0, k3, k2, k1]*W4_tensor[k0, k1, k2, k3] * \
            W4_tensor[k2, k1, k0, k3]*W4_tensor[k2, k3, k0, k1] * \
            (ν4_bras[k3]*μ4_kets[k2])*(μ4_bras[k2]*ν4_kets[k1]) * \
            (ν4_bras[k1]*μ4_kets[k0])*(ν4_kets[k3]*μ4_bras[k0])  
            for k3 in range(len(d)) \
            for k2 in range(len(c)) \
            for k1 in range(len(b)) \
            for k0 in range(len(a)) \
            )
print("%s seconds" % (time.time() - start_time))

13.722368001937866 seconds


In [650]:
VTA4 = Qobj(VTA4_3, dims = [[2]*4, [2]*4])
VTA4_1 == VTA4_2 == VTA4_3

True

In [ ]:
# define operators for bosonic modes 
a = destroy(cutoff)
Ib = identity(cutoff)

# define vacuum state for a cavity, and spin up and spin down states for qubits
vacuum = basis(cutoff, 0)
up = basis(2,0)
down = basis(2,1)

# define projection operators that project onto vacuum either cavity 1 
# or cavities 1 and 2 
vacuum_projector = vacuum*vacuum.dag()
P_vac1 = tensor(vacuum_projector, Ib, I2, I2, I2)
P_vac12 = tensor(vacuum_projector, vacuum_projector, I2, I2, I2) 

# create array over which to sweep α
α_array = np.linspace(α_start, α_end, α_steps)

In [ ]:
[expect(ρ_ground_state3, ((P_vac12*GSP(α)*initial_state).unit()).ptrace([2,3,4])) for α in α_array]

In [599]:
ρ_ground_state4

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.08333333  0.         -0.16666667
   0.08333333  0.          0.          0.08333333 -0.16666667  0.
   0.08333333  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0. 

In [635]:
def generate_initial_states(N_cavities, N_qubits, num_states, cutoff):
    
    '''
    generate list of initial states which is a product of state of 
    the number of cavities and qubits specified by the arguments
    '''
    
    if N_cavities == 0 and cutoff == 0: 
        return [tensor([rand_ket(2) for i in range(N_qubits)]) \
                for initial_state in range(num_states)]
    else:

        # returns a product state of (N_cavities) cavities and (N_qubits) 
        # random qubits for specified number of initial states
        return [tensor([basis(cutoff, 0)]*N_cavities + [rand_ket(2) \
                for i in range(N_qubits)]) \
                for initial_state in range(num_states)]

In [639]:
N_cavities = 0 
N_qubits = 4
num_states = 1
cutoff = 0

Ψ0_list = generate_initial_states(N_cavities, N_qubits, num_states, cutoff)

In [663]:
(VTA4*Ψ0_list[0]).unit()

Quantum object: dims = [[2, 2, 2, 2], [1, 1, 1, 1]], shape = (16, 1), type = ket
Qobj data =
[[ 0.        +0.j        ]
 [-0.13433085-0.19447174j]
 [-0.06778627-0.098341j  ]
 [ 0.10597027+0.25650272j]
 [ 0.13433082+0.19447166j]
 [-0.0398828 -0.02387889j]
 [-0.07007484-0.17143329j]
 [ 0.07718492+0.00221921j]
 [ 0.0677863 +0.09834108j]
 [ 0.33776413+0.27783849j]
 [-0.03988287-0.02387903j]
 [-0.26388735-0.32857664j]
 [-0.29389389-0.31515j   ]
 [-0.07718497-0.00221926j]
 [ 0.26388739+0.32857669j]
 [ 0.        +0.j        ]]

# $\text{Conversion to Julia code }$

In [567]:
# Convert the NumPy array of arrays to a list of lists
μ4_kets_complex_list = [eigvec.full().tolist() for eigvec in μ4_kets]
μ4_bras_complex_list = [eigvec.full().tolist() for eigvec in μ4_bras]
ν4_kets_complex_list = [eigvec.full().tolist() for eigvec in ν4_kets]
ν4_bras_complex_list = [eigvec.full().tolist() for eigvec in ν4_bras]

In [568]:
# collect only the real part of each entry if imaginary part is 0
μ4_kets_list = [[x[0].real if x[0].imag == 0 else x[0] for x in vec] \
                    for vec in μ4_kets_complex_list]
μ4_bras_list = [[x[0].real if x[0].imag == 0 else x[0] for x in vec] \
                    for vec in μ4_bras_complex_list]
ν4_kets_list = [[x[0].real if x[0].imag == 0 else x[0] for x in vec] \
                    for vec in ν4_kets_complex_list]
ν4_bras_list = [[x[0].real if x[0].imag == 0 else x[0] for x in vec] \
                    for vec in ν4_bras_complex_list]

In [591]:
# specificy directory into which we will store data
path = os.getcwd() + '/four_site_data/'

# save list of eigenvalues 
spectra4_list = [x.tolist() for x in spectra4]
with open(path+"spectra4.json", "w") as json_file:
    json.dump(spectra4_list, json_file)

# Save each list to a JSON file
with open(path+"μ4_kets.json", "w") as json_file:
    json.dump(μ4_kets_list, json_file)
    
with open(path+"μ4_bras.json", "w") as json_file:
    json.dump(μ4_kets_list, json_file)
    
with open(path+"ν4_kets.json", "w") as json_file:
    json.dump(μ4_kets_list, json_file)
    
with open(path+"ν4_bras.json", "w") as json_file:
    json.dump(μ4_kets_list, json_file)

# Additional Code

In [ ]:
def weight_tensor(α, N, eigvals_array, indices):
    
    # convert list of indices into an array
    # and calculate length of each eigenvalues array
    index_array = np.array(indices)
    d = 2**N - 1
    
    # check whether all indices are >= 0 and <= d
    all(np.logical_and(index_array >= 0, index_array <= d))
    
    # collect the sum of the eigenvalues specified by indices
    eigval_sum = 0 
    for eigval_list, index in zip(eigvals_array, index_array): 
        eigval_sum += eigval_list[index]
        
    return np.exp(-((α**2)/2)*(eigval_sum**2))    

In [ ]:
# find eigenergies and eigenstates of local Hamiltonians
H12_eigenvalues, H12_eigenstates = H4_list[0].eigenstates()
H23_eigenvalues, H23_eigenstates = H4_list[1].eigenstates()
H34_eigenvalues, H34_eigenstates = H4_list[2].eigenstates()
H41_eigenvalues, H41_eigenstates = H4_list[3].eigenstates()

# construct local Hamiltonians through spectral decomposition
H12 = sum([λ*vec*vec.dag() for λ, vec in zip(H12_eigenvalues, H12_eigenstates)])
H23 = sum([λ*vec*vec.dag() for λ, vec in zip(H23_eigenvalues, H23_eigenstates)])
H34 = sum([λ*vec*vec.dag() for λ, vec in zip(H34_eigenvalues, H34_eigenstates)])
H41 = sum([λ*vec*vec.dag() for λ, vec in zip(H41_eigenvalues, H41_eigenstates)])

# display equalities
display(H12 == H4_list[0])
display(H23 == H4_list[1])
display(H34 == H4_list[2])
display(H41 == H4_list[3])

In [ ]:
# convert lists of eigenvectors into arrays 
H12_eigenstates_array = [np.array(eigenstate) for eigenstate in H12_eigenstates]
blue_eigvecs_array = [np.array(eigenstate) for eigenstate in blue_eigvecs_list]